## Input Connectors -> ConfluentKafka Input

1. prepare the kafka cluster

In [4]:
%%bash
# starting kafka container
docker compose -f ../../../../../examples/compose/docker-compose.yml down -v  
docker compose -f ../../../../../examples/compose/docker-compose.yml up -d kafka
# creating the topic
docker exec -i kafka /bin/bash -c "/opt/bitnami/kafka/bin/kafka-topics.sh --create --topic consumer --bootstrap-server localhost:9092 --partitions 1 --replication-factor 1"
# waiting for kafka to be ready
sleep 10


bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
 Container kafka  Stopping
 Container kafka  Stopped
 Container kafka  Removing
 Container kafka  Removed
 Network compose_kafka  Removing
 Network compose_kafka  Removed
 Network compose_kafka  Creating
 Network compose_kafka  Created
 Container kafka  Creating
 Container kafka  Created
 Container kafka  Starting
 Container kafka  Started
[2025-08-22 07:57:37,132] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Node may not be available. (org.apache.kafka.clients.NetworkClient)
[2025-08-22 07:57:37,237] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Node may not be available. (org.apache.kafka.clients.NetworkClient)
[2025-08-22 07:57:37,338] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Node may not be availabl

Created topic consumer.


2. produce messages

In [6]:
%%bash
# producing 3 events to kafka topic consumer
docker exec -i kafka /bin/bash -c "echo '{\"message\": \"the message\"}' | /opt/bitnami/kafka/bin/kafka-console-producer.sh --bootstrap-server localhost:9092 --topic consumer "
# showing events in kafka
# docker exec -i kafka /bin/bash -c "/opt/bitnami/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic consumer --from-beginning --max-messages 10"


bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


3. initializing the confluentkafka_input connector

In [7]:
import sys
import logging
from logprep.factory import Factory
from logprep.ng.connector.confluent_kafka.input import ConfluentKafkaInput

# Configure logging
logging.basicConfig(
    level=logging.DEBUG, 
    stream=sys.stdout
)

# Create a Kafka input connector configuration

kafka_config = {
    "kafka": {
        "type": "ng_confluentkafka_input",
        "topic": "consumer",
        "kafka_config": {
            "bootstrap.servers": "127.0.0.1:9092",
            "group.id": "cgroup3",
            "enable.auto.commit": "true",
            "auto.commit.interval.ms": "10000",
            "enable.auto.offset.store": "false",
            "queued.min.messages": "100000",
            "queued.max.messages.kbytes": "65536",
            "statistics.interval.ms": "60000"
        }
    }
}

kafka_connector: ConfluentKafkaInput = Factory.create(kafka_config)

# Start the connector
kafka_connector.setup()

# show the current backlog
print(f"{kafka_connector.event_backlog.backlog=}")

# Consume 3 messages from the Kafka topic
event = next(kafka_connector(timeout=10))
event = next(kafka_connector(timeout=10))
event = next(kafka_connector(timeout=10))

# show the backlog after consuming an event
print(f"{kafka_connector.event_backlog.backlog=}")

# Shut down the connector to close the producer
kafka_connector.shut_down()


DEBUG:Component:Checking health of kafka
kafka_connector.event_backlog.backlog=set()
INFO:KafkaInput:bs0010tnu-devclient-PipelineNone-pid72104-af11cefa-25a5-4d9f-9ad2-ca501676a9d6 was assigned to topic: consumer | partition 0
kafka_connector.event_backlog.backlog={LogEvent(data={'message': 'the message'}, state=received)}
